In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm

In [3]:
df = pd.read_csv("data.csv")

metadata_cols = ["country", "currency", "price_perception", "average_rental_price"]
metadata = df[metadata_cols]
metadata = metadata.values.repeat(30, axis=0)

prices_cols = ["kg", "price"]
prices = df.values[:, 8:].reshape(-1, 2)

products = np.array(["apples", "bananas", "tomatoes", "potatoes", "flour", "rice", "milk", "butter", "eggs", "chicken"])
products = products.repeat(3).reshape(-1, 1).repeat(50, axis=1).T.reshape(-1, 1)

df = pd.DataFrame()
df[metadata_cols] = metadata
df["product"] = products
df[prices_cols] = prices

df = df.dropna()
df.head()

,country,currency,price_perception,average_rental_price,product,kg,price
0,Germany,EUR,Budget (cheap),826,apples,2.0,2.22
1,Germany,EUR,Budget (cheap),826,apples,1.0,2.99
2,Germany,EUR,Budget (cheap),826,apples,1.0,1.49
3,Germany,EUR,Budget (cheap),826,bananas,1.0,3.9
4,Germany,EUR,Budget (cheap),826,bananas,1.0,1.59


In [4]:
# standardize city names
replacements = [
    [["berlin", "germany"], "berlin"],
    [["kingdom", "kindom", "uk", "london"], "london"],
    [["seoul"], "seoul"],
    [["states"], "san francisco"],
]

for unique_val in df.country.unique():
    for matches, replacement in replacements:
        if any(s in unique_val.lower() for s in matches):
            df.country.replace(unique_val, replacement, inplace=True)

print(df.country.unique())

# shorten names
df.replace({"price_perception": {"Budget (cheap)": "cheap", "Mid-range": "mid", "Luxury (expensive)": "high"}}, inplace=True)
df.rename(columns={"price_perception": "brand", "average_rental_price": "rent", "country": "city"}, inplace=True)

['berlin' 'seoul' 'london' 'san francisco']


In [5]:
# source: google
euro_per_dollar = 0.92
won_per_dollar = 1232
gbp_per_dollar = 0.76

currency_conversion = np.ones(len(df))
currency_conversion[df["currency"] == "EUR"] = euro_per_dollar
currency_conversion[df["currency"] == "GBP"] = gbp_per_dollar
currency_conversion[df["currency"] == "KRW"] = won_per_dollar

# perform currency adjustment
df["price_usd"] = df["price"] / currency_conversion

# control for quantity
df["dpkg"] = df["price_usd"] / df["kg"]
df = df[["product", "city", "brand", "rent", "dpkg"]]

df.head()

,product,city,brand,rent,dpkg
0,apples,berlin,cheap,826,1.206522
1,apples,berlin,cheap,826,3.25
2,apples,berlin,cheap,826,1.619565
3,bananas,berlin,cheap,826,4.23913
4,bananas,berlin,cheap,826,1.728261


In [11]:
cities = pd.get_dummies(df.city)
brands = pd.get_dummies(df.brand)
products = pd.get_dummies(df["product"])

(1326, 4)

In [12]:
with pm.Model() as model:
    # random variables represent unknown price multipliers
    city_coeffs = pm.Gamma("city_coeffs", alpha=1, beta=1, shape=cities.shape[1])
    brand_coeffs = pm.Gamma("brand_coeffs", alpha=1, beta=1, shape=brands.shape[1])
    product_coeffs = pm.Gamma("product_coeffs", alpha=1, beta=1, shape=products.shape[1])

    # calculate effects for our data
    city_effects = cities.values @ city_coeffs
    brand_effects = brands.values @ brand_coeffs
    product_effects = products.values @ product_coeffs

    # add gaussian noise
    observation_noise = pm.Gamma("price_noise", 1, 1)
    price_observed = pm.Normal(
        "observed_price",
        mu=city_effects*brand_effects*product_effects,
        sigma=observation_noise,
        observed=df.dpkg.values
    )

    # perform sampling
    prior = pm.sample_prior_predictive()
    posterior = pm.sample()
    posterior_pred = pm.sample_posterior_predictive(posterior)


You can find the C code in this temporary file: /var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/theano_compilation_error_oyc2filc
library imit=] is not found.


Exception: ('Compilation failed (return status=1): In file included from /Users/leoware/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.5-64/tmpfdn5d44_/mod.cpp:1:. /usr/local/Caskroom/miniconda/base/include/python3.9/Python.h:14:2: error: "Something\'s broken.  UCHAR_MAX should be defined in limits.h.". #error "Something\'s broken.  UCHAR_MAX should be defined in limits.h.".  ^. /usr/local/Caskroom/miniconda/base/include/python3.9/Python.h:18:2: error: "Python\'s source code assumes C\'s unsigned char is an 8-bit type.". #error "Python\'s source code assumes C\'s unsigned char is an 8-bit type.".  ^. /usr/local/Caskroom/miniconda/base/include/python3.9/Python.h:27:5: error: "Python.h requires that stdio.h define NULL.". #   error "Python.h requires that stdio.h define NULL.".     ^. In file included from /Users/leoware/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.5-64/tmpfdn5d44_/mod.cpp:1:. In file included from /usr/local/Caskroom/miniconda/base/include/python

In [13]:
X = np.random.random([100, 3])
y = -2*X[:,0] + np.random.normal(0, 0.2)

with pm.Model() as linear_model:
    weights = pm.Normal("weights", mu=0, sigma=1, shape=(3))
    noise = pm.Gamma("noise", alpha=2, beta=1)
    y_observed = pm.Normal(
        "y_observed",
        mu=X @ weights,
        sigma=noise,
        observed=y,
    )

    prior = pm.sample_prior_predictive()
    posterior = pm.sample()
    posterior_pred = pm.sample_posterior_predictive(posterior)


You can find the C code in this temporary file: /var/folders/l_/zjyw1l0d37x3lbbq6x4vvs_00000gn/T/theano_compilation_error_lru83j8f
library imit=] is not found.


Exception: ('Compilation failed (return status=1): In file included from /Users/leoware/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.5-64/tmpezdjblws/mod.cpp:1:. /usr/local/Caskroom/miniconda/base/include/python3.9/Python.h:14:2: error: "Something\'s broken.  UCHAR_MAX should be defined in limits.h.". #error "Something\'s broken.  UCHAR_MAX should be defined in limits.h.".  ^. /usr/local/Caskroom/miniconda/base/include/python3.9/Python.h:18:2: error: "Python\'s source code assumes C\'s unsigned char is an 8-bit type.". #error "Python\'s source code assumes C\'s unsigned char is an 8-bit type.".  ^. /usr/local/Caskroom/miniconda/base/include/python3.9/Python.h:27:5: error: "Python.h requires that stdio.h define NULL.". #   error "Python.h requires that stdio.h define NULL.".     ^. In file included from /Users/leoware/.theano/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.9.5-64/tmpezdjblws/mod.cpp:1:. In file included from /usr/local/Caskroom/miniconda/base/include/python